# 语言模型
------
## 1 简介
- pytorch实现LSTM训练语言模型
- 使用的数据集：
    - bobsue.lm.train.txt：语言模型训练数据（LMTRAIN）
    - bobsue.lm.dev.txt：语言模型验证数据（LMDEV）
    - bobsue.lm.test.txt：语言模型测试数据（LMTEST）
    - bobsue.prevsent.train.tsv：基于上文的语言模型训练数据（PREVSENTTRAIN）
    - bobsue.prevsent.dev.tsv：基于上文的上文语言模型验证数据（PREVSENTDEV）
    - bobsue.prevsent.test.tsv：基于上文的上文语言模型测试数据（PREVSENTTEST）
    - bobsue.voc.txt：词汇表文件，每行是一个单词
    - lm文件中的每一行都包含一个故事中的句子。prev文件中的每一行都包含一故事中的一个句子，tab，然后是故事中的下一个句子。注意：prevsent文件中每一行的第二个字段与相应的lm文件中的对应行相同。( 也就是说：cut -f 2 bobsue.prevsent.x.tsv与bobsue.lm.x.txt相同）完整的词汇表包含在文件bobsue.voc.txt中，每一行是一个单词。在这个任务中不会出现未知单词。
- 评估
    - 我们使用单词预测准确率作为主要评估指标而非困惑度(perplexity)。因为当你试图比较某些损失函数时，perplexity不太好用。

-----
## 2 具体内容
### 用Log Loss训练LSTM模型
- 实现一个基于LSTM的语言模型。具体为：
    - 对每个当前的hidden state做一个线性变化和softmax处理，预测下一个单词。
    - 使用Log Loss (Cross Entropy Loss)来训练该模型。使用**EVALLM**的步骤来评估模型。
    - 汇报模型训练结果和代码。你的单词预测准确率应该能够达到30%以上。
- 要求
    - 至少训练10个epoch
    - 可以使用不同的模型参数。建议使用一层LSTM，200 hidden dimension作为词向量和LSTM hidden state的大小。
    - 模型参数的初始值可以随机设定
    - 输入和输出层的word embedding参数可以不一样，当然你也可以尝试把他们设置成一样。
    - 使用Adam或者SGD等optimizer来优化模型。
    - 在提交报告的时候请尽可能详细描述你的所有模型参数。
----
### 错误分析
- 请在你的代码中添加一项功能，可以展示出你模型预测错误的单词，将标准答案单词和模型预测的单词分别打印出来。
- 请写下你的模型最常见的35个预测错误(正确答案是a，模型预测了b)。
- 通过观察这些常见的错误，将错误分类。你不需要将每个错误都分类，不过建议同学们花点时间观察自己模型的错误，看看他们是否有一定的相关性。大家可以尝试从以下角度思考错误类型：
    - 为什么你的模型会预测出这个单词？
    - 模型怎么样才能做得更好？这个模型犯的错误是很接近正确答案的吗？如果是的话，这个错误答案与正确答案有何相似之处？
    - 把这35个预测错误归类成你定义的错误类别。讨论一下你的模型在哪些方面做得比较好，哪些方面做的不好。

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from collections import Counter

In [2]:
# 数据文件
word_file = './data/bobsue.voc.txt'
train_file = './data/bobsue.lm.train.txt'
test_file = './data/bobsue.lm.test.txt'
dev_file = './data/bobsue.lm.dev.txt'

BATCH_SIZE = 32       # 批次大小
EMBEDDING_DIM = 200   # 词向量维度
HIDDEN_DIM = 200      # 隐含层
GRAD_CLIP = 5.        # 梯度截断值
EPOCHS = 20 
LEARN_RATE = 0.01     # 初始学习率

BEST_VALID_LOSS = float('inf')     # 初始验证集上的损失值，设为最大
MODEL_PATH = "lm-best-dim{}.pth"   # 模型名称
USE_CUDA = torch.cuda.is_available()    # 是否使用GPU
NUM_CUDA = torch.cuda.device_count()    # GPU数量

## 1 数据预处理
### 1.1 读取数据文件，构建词汇集、word2idx、idx2word。

In [3]:
def load_word_set(filename):
    with open(filename, "r", encoding="utf-8") as f:
        word_set = set([line.strip() for line in f])
    return word_set

In [4]:
word_set = load_word_set(word_file)
word2idx = {w:i for i, w in enumerate(word_set, 1)}
idx2word = {i:w for i, w in enumerate(word_set, 1)}

# 将pad的索引设置为0并添加到词表
PAD_IDX = 0
word2idx["<pad>"] = PAD_IDX
idx2word[PAD_IDX] = "<pad>"

### 1.2 训练、验证、测试数据准备
- 将数据处理成模型可以接收的格式

In [5]:
def load_corpus(filename):
    """读取数据集，返回句子列表"""
    with open(filename, "r", encoding="utf-8") as f:
        sentences = [line.strip() for line in f]
    return sentences

def sentences2words(sentences):
    """将句子列表转换成单词列表"""
    return [w for s in sentences for w in s.split()]

def max_sentence_num(sentences):
    """返回最长句子单词数量"""
    return max([len(s.split()) for s in sentences ])

In [6]:
# 各数据集的句子列表
train_sentences = load_corpus(train_file)
dev_sentences = load_corpus(dev_file)
test_sentences = load_corpus(test_file)

# 各数据集的单词列表
train_words = sentences2words(train_sentences)
dev_words = sentences2words(dev_sentences)
test_words = sentences2words(test_sentences)

In [7]:
# 查看处理后训练集、验证集、测试集的基本情况
s = "{}句子数: {}, 单词数: {}."
print(s.format("训练集", len(train_sentences), len(train_words)))
print(s.format("验证集", len(dev_sentences), len(dev_words)))
print(s.format("测试集", len(test_sentences), len(test_words)))

print("-"*50)

# 这里需要知道各数据集上最长句子的单词📚，以便后面构造单词索引向量的时候设置一个恰当的维度
print("训练集最长句子单词个数：", max_sentence_num(train_sentences))
print("验证集最长句子单词个数：", max_sentence_num(dev_sentences))
print("测试集最长句子单词个数：", max_sentence_num(test_sentences))

训练集句子数: 6036, 单词数: 71367.
验证集句子数: 750, 单词数: 8707.
测试集句子数: 750, 单词数: 8809.
--------------------------------------------------
训练集最长句子单词个数： 21
验证集最长句子单词个数： 20
测试集最长句子单词个数： 21


In [8]:
def build_x_y(corpus, word2idx, seq_len=21):
    """
    构造输入模型的特征以及标签。
    输入：
        corpus： 列表，每个元素是一个句子。
        word2idx： 字典，key是单词，value是单词的索引。
        seq_len：int, 句子切分后的单词序列的长度。
    返回：
        sentences：二维列表，每一行是一个句子切分后单词的索引列表（不包括句子的最后一个单词）。输入模型的x。
        labels：二维列表，每一行是一个句子切分后单词的索引列表（不包括句子的第一个单词）。y。
    """
    sentences = []
    labels = []
    for sentence in corpus:
        words = sentence.split()
        sentence_vec = [0]*seq_len
        for i, w in enumerate(words[:-1]):
            sentence_vec[i] = word2idx[w]
        sentences.append(sentence_vec)
        
        label_vec = [0] * seq_len
        for i, w in enumerate(words[1:]):
            label_vec[i] = word2idx[w]
        labels.append(label_vec)
    return sentences, labels

In [9]:
train_data, train_label = build_x_y(train_sentences, word2idx)
dev_data, dev_label = build_x_y(dev_sentences, word2idx)
test_data, test_label = build_x_y(test_sentences, word2idx)

In [10]:
# 查看处理后的训练集第一个样本及标签
print(train_data[1])
print(" "*5, train_label[1])

[1075, 156, 471, 192, 460, 484, 943, 340, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
      [156, 471, 192, 460, 484, 943, 340, 534, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [11]:
idx = 1
print(" ".join([idx2word[i] for i in train_data[idx]]))

print(" "*3, " ".join([idx2word[i] for i in train_label[idx]]))

<s> The girl broke up with Bob . <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>
    The girl broke up with Bob . </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>


In [12]:
# 构造批次数据
def build_batch_data(data, label, batch_size=32):
    """构建 batch tensor，返回 batch 列表，每个batch为二元组包含data和label"""
    batch_data = []
    data_tensor = torch.tensor(data, dtype=torch.long)
    label_tensor = torch.tensor(label, dtype=torch.long)
    n, dim = data_tensor.size()
    for start in range(0, n, batch_size):
        end = start + batch_size
        if end > n:
            dbatch = data_tensor[start: ]
            lbatch = label_tensor[start: ]
            print("最后一个batch size:", dbatch.size())
            break
        else:
            dbatch = data_tensor[start: end]
            lbatch = label_tensor[start: end]
        batch_data.append((dbatch, lbatch))
    return batch_data

In [13]:
train_batch = build_batch_data(train_data, train_label, batch_size=BATCH_SIZE)
dev_batch = build_batch_data(dev_data, dev_label, batch_size=BATCH_SIZE)
test_batch = build_batch_data(test_data, test_label, batch_size=BATCH_SIZE)

最后一个batch size: torch.Size([20, 21])
最后一个batch size: torch.Size([14, 21])
最后一个batch size: torch.Size([14, 21])


In [14]:
# 查看各数据集有多少个batch
print(len(train_batch), len(dev_batch), len(test_batch))

188 23 23


## 2 定义模型

In [15]:
# 定义模型
class MyLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(MyLSTM, self).__init__()
        self.vocab_size = vocab_size
        self.hidden_dim = hidden_dim
        self.word_embeddings = nn.Embedding(self.vocab_size, embedding_dim)
        # batch_first=True 意味着输入是(batch, seq, feature)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.hidden2word = nn.Linear(hidden_dim, self.vocab_size)
        
    def forward(self, x):
        embeds = self.word_embeddings(x)
        lstm_out, (h_n, c_n) = self.lstm(embeds)
        target_space = self.hidden2word(lstm_out.contiguous().view(-1, self.hidden_dim))
        mask = (x != PAD_IDX).view(-1)
        mask_target = target_space[mask]
        
        target_scores = F.log_softmax(mask_target, dim=1)
        return target_scores

    
# 计算准确率
def acc_score(pred_score, y):
    # 返回最大的概率的索引
    y_pred = pred_score.argmax(dim=1)
    # print(y.view(-1))
    acc_count = torch.eq(y_pred, y.view(-1))
    score = acc_count.sum().item() / acc_count.size()[0]
    return score


# 训练函数
def train(model, device, iterator, optimizer, criterion, grad_clip):
    epoch_loss = 0  # 积累变量
    epoch_acc = 0   # 积累变量
    model.train()   # 该函数表示PHASE=Train
    
    for x, y in iterator:  # 拿每一个minibatch
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
        mask = y != PAD_IDX
        pure_y = y[mask]
        
        fx = model(x)                 # 进行forward
        loss = criterion(fx, pure_y)  # 计算loss
        acc = acc_score(fx, pure_y)   # 计算准确率
        loss.backward()               # 进行BP
        
        # 梯度裁剪
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        optimizer.step()  # 更新参数
        
        epoch_loss += loss
        epoch_acc += acc
        
    return epoch_loss/len(iterator),epoch_acc/len(iterator)


# 验证函数，验证集和测试集用，不更新梯度
def evaluate(model, device, iterator, criterion):
    model.eval()  # 不更新参数，预测模式
    epoch_loss=0  # 积累变量
    epoch_acc=0   # 积累变量
    
    with torch.no_grad():
        for x, y in iterator:
            x = x.to(device)
            y = y.to(device)
            mask = y != PAD_IDX
            pure_y = y[mask]
            
            fx = model(x)
            loss = criterion(fx, pure_y)
            acc = acc_score(fx, pure_y)
            epoch_loss += loss
            epoch_acc += acc
    return epoch_loss/len(iterator), epoch_acc/len(iterator)

## 3 模型训练与评价

In [16]:
VOCAB_SIZE = len(word2idx)     # 词汇表长度

model = MyLSTM(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM)

DEVICE = torch.device("cuda" if USE_CUDA else 'cpu')
model = model.to(DEVICE)

# 使用多块GPU
if NUM_CUDA > 1:
    device_ids = list(range(NUM_CUDA))
    print(device_ids)
    model = nn.DataParallel(model, device_ids=device_ids)
    # model = nn.parallel.DistributedDataParallel(model, device_ids=device_ids)

[0, 1, 2, 3]


- 保存最优模型的逻辑是，每一个epoch之后再对比验证集损失值，验证集损失降低才认为模型更优。

In [18]:
criterion = nn.NLLLoss()                                             # 指定损失函数
optimizer = optim.Adam(model.parameters(), lr=LEARN_RATE)            # 指定优化器
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.5)   # 学习率缩减

SCHED_NUM = 0
for epoch in range(1, EPOCHS+1):
    train_loss, train_acc = train(model, DEVICE, train_batch, optimizer, criterion, GRAD_CLIP)
    valid_loss, valid_acc = evaluate(model, DEVICE, dev_batch, criterion)
    if valid_loss < BEST_VALID_LOSS: # 如果是最好的模型就保存到文件夹
        BEST_VALID_LOSS = valid_loss
        torch.save(model, MODEL_PATH.format(EMBEDDING_DIM))
        SCHED_NUM = 0
    else:
        SCHED_NUM += 1
        if SCHED_NUM % 3 == 0:
            scheduler.step()
            print("Current lr:", optimizer.param_groups[0]['lr'])
        if SCHED_NUM == 7:
            print("Early stop!")
            break
    print('Epoch:{}|Train Loss:{:.4}|Train Acc:{:.4}|Val Loss:{:.4}|Val Acc:{:.4}'.format(epoch,train_loss,train_acc,valid_loss,valid_acc))

Epoch:1|Train Loss:3.447|Train Acc:0.32|Val Loss:3.509|Val Acc:0.3222
Epoch:2|Train Loss:2.995|Train Acc:0.3483|Val Loss:3.574|Val Acc:0.3212
Epoch:3|Train Loss:2.715|Train Acc:0.374|Val Loss:3.663|Val Acc:0.3158
Current lr: 0.01
Epoch:4|Train Loss:2.507|Train Acc:0.4024|Val Loss:3.785|Val Acc:0.3186
Epoch:5|Train Loss:2.326|Train Acc:0.4318|Val Loss:3.907|Val Acc:0.3104
Epoch:6|Train Loss:2.167|Train Acc:0.4622|Val Loss:4.015|Val Acc:0.3077
Current lr: 0.005
Epoch:7|Train Loss:2.045|Train Acc:0.486|Val Loss:4.12|Val Acc:0.3048
Early stop!


In [19]:
model = torch.load(MODEL_PATH.format(EMBEDDING_DIM))
test_loss, test_acc = evaluate(model, DEVICE, test_batch, criterion)
print('Test Loss: {0} | Test Acc: {1} |'.format(test_loss, test_acc))

Test Loss: 3.564551591873169 | Test Acc: 0.31922278920502517 |


## 4 打印错误单词

In [20]:
# 答应预测错误的单词
def print_pred_error_words(model,device,data_batch):
    model.eval()
    error_words = []
    with torch.no_grad():
        for x, y in data_batch:
            x = x.to(device)
            y = y.to(device)
            
            mask = (y!=PAD_IDX)
            fx = model(x)
            
            pred_idx = fx.argmax(dim=1)
            ground_truth_idx = y[mask]
            for p, g in zip(pred_idx.tolist(), ground_truth_idx.tolist()):
                if p != g:
                    error_words.append(" | ".join([idx2word[g], idx2word[p]]))
    return error_words

In [21]:
model = torch.load(MODEL_PATH.format(EMBEDDING_DIM))
error_words = print_pred_error_words(model, DEVICE, test_batch)

In [22]:
words_counter = Counter(error_words)
TopN = 35
topn_words = words_counter.most_common(TopN)
print("真实值 | 预测值 | 预测错误次数")
for w in topn_words:
    print(w)

真实值 | 预测值 | 预测错误次数
('Bob | He', 137)
('She | He', 109)
('Sue | He', 89)
('and | .', 49)
('had | was', 45)
('decided | was', 43)
('to | .', 42)
('her | the', 34)
('his | the', 33)
('in | .', 30)
(', | .', 30)
('she | he', 28)
('for | .', 28)
('His | He', 26)
('. | to', 26)
('One | He', 25)
('. | the', 24)
('a | the', 24)
('But | He', 21)
('Her | He', 21)
('The | He', 21)
('When | He', 19)
('went | was', 19)
('They | He', 19)
('got | was', 17)
('he | Bob', 17)
('wanted | was', 17)
('! | .', 16)
('on | .', 16)
('he | to', 16)
('at | .', 16)
("'s | was", 15)
('the | .', 15)
('Sue | Bob', 15)
('it | the', 14)


## 5 报告
------
### 模型参数
- BATCH_SIZE = 32       # 批次大小
- EMBEDDING_DIM = 200   # 词向量维度
- HIDDEN_DIM = 200      # 隐含层
- GRAD_CLIP = 5.        # 梯度截断值
- EPOCHS = 20           # epoch
- LEARN_RATE = 0.01     # 初始学习率
### 各指标
- 训练集准确率为32%， 验证集准确率为32.22%，测试集准确率为31.92%。
### 错误分析
- 请在你的代码中添加一项功能，可以展示出你模型预测错误的单词，将标准答案单词和模型预测的单词分别打印出来。
    - 答：见代码
- 请写下你的模型最常见的35个预测错误(正确答案是a，模型预测了b)。
```
真实值 | 预测值 | 预测错误次数
('Bob | He', 137)
('She | He', 109)
('Sue | He', 89)
('and | .', 49)
('had | was', 45)
('decided | was', 43)
('to | .', 42)
('her | the', 34)
('his | the', 33)
('in | .', 30)
(', | .', 30)
('she | he', 28)
('for | .', 28)
('His | He', 26)
('. | to', 26)
('One | He', 25)
('. | the', 24)
('a | the', 24)
('But | He', 21)
('Her | He', 21)
('The | He', 21)
('When | He', 19)
('went | was', 19)
('They | He', 19)
('got | was', 17)
('he | Bob', 17)
('wanted | was', 17)
('! | .', 16)
('on | .', 16)
('he | to', 16)
('at | .', 16)
("'s | was", 15)
('the | .', 15)
('Sue | Bob', 15)
('it | the', 14)
```
- 通过观察这些常见的错误，将错误分类。你不需要将每个错误都分类，不过建议同学们花点时间观察自己模型的错误，看看他们是否有一定的相关性。大家可以尝试从以下角度思考错误类型：
    - 为什么你的模型会预测出这个单词？
    - 模型怎么样才能做得更好？这个模型犯的错误是很接近正确答案的吗？如果是的话，这个错误答案与正确答案有何相似之处？
    - 把这35个预测错误归类成你定义的错误类别。讨论一下你的模型在哪些方面做得比较好，哪些方面做的不好。
    
 - 答：
    - 错误分类：
    - (1)将人名预测成代词，比如Bob、Sue预测成龙He。不仅如此，模型将很多行首的词都预测成了He，这可能是训练的时候行首的单词之前没有足够多的信息，如果我们将句子双向输入或者反向输入进行训练可能会改善这种情况。
    - (2)代词预测错误，分不清男女。She预测成He；she预测成he等，这种错误相对比较接近，错误答案和正确答案都是人称代词。
    - (3)连词预测错误，存在and预测成句号，But预测成He。
    - (4)标点预测错误，逗号预测成句号，感叹号预测成句号。错误答案和正确答案相近。
    - (5)介词预测错误。to/in/for/at等都有被预测成了句号。错误答案和正确答案并不接近，模型可能对介词预测不好。之所以存在这种问题，可能是因为原来的句子中to/in/for/at 之前的单词已经能构成一个完整句子，所以模型可能认为前面的单词已经能构成完整句子了，直接就将介词预测成标点了。
    - 模型做得比较好的地方就是句子的中间部分后结尾部分预测的准确率高一些，行首预测的准确率低一些，这可能是因为训练是从行首开始输入，没有学到之前的信息。进行双向输入训练可能改善这种情况。此外，对介词预测不好，模型可能认为介词之前的句子结构完整，故会把介词预测成标点符号。